In [1]:
import os
import json
from utils import error_analysis_event

In [ ]:
{
        "experiment_dir": "../experiments/MUC/g2_environments/keyword/default",
        "output_dir": "../experiments/MUC/error_analysis/keyword/default",
        "hungarian_config": "scorer_configs/muc_config.json",
        "types_mapping": "types_mapping/types_mapping_muc.json",
        "gtt_ref_dev": "../data/MUC/gtt/keyword/dev.json",
        "tanl_ref_dev": "../data/MUC/keyword/dev_event.json",
        "gtt_ref_test": "../data/MUC/gtt/keyword/test.json",
        "tanl_ref_test": "../data/MUC/keyword/test_event.json",
        "relax_match": true,
        "dataset_name": "muc"
    }

In [2]:
best_epochs = {
    'keyword': {
        'default': 26,
        'single_pass': 93,
        'args_only': 31
    },
    'llm': {
        'default': 5,
        'single_pass': 114,
        'args_only': 93
    },
    'human': {
        'default': 4,
        'single_pass': 111,
        'args_only': 32
    },
    'no_trig': {
        'no_trig': 93
    }
}
roles = ["PerpInd", "PerpOrg", "Target", "Victim", "Weapon"]

In [4]:
def create_directories_for_file(file_path):
    directory_path = os.path.dirname(file_path)
    folders = directory_path.split(os.sep)
    path_to_create = ""
    for folder in folders:
        path_to_create = os.path.join(path_to_create, folder)
        if not os.path.exists(path_to_create):
            os.makedirs(path_to_create)

for split_style in os.listdir('../bucketed/MUC'):
    # for trigger_source in ['keyword', 'human', 'llm']:
    #     for experiment in ['default', 'single_pass', 'args_only']:
    for trigger_source in ['no_trig']:
        for experiment in ['no_trig']:
            best_epoch = best_epochs[trigger_source][experiment]
            model_out = f'../experiments/MUC/g2_environments/{trigger_source}/{experiment}/test_predictions/{best_epoch}.0.txt'
            tanl_ref = f'../data/MUC/{trigger_source}/test_event.json'
            # gtt_ref = f'../data/MUC/gtt/{trigger_source}/test.json'
            gtt_ref = f'../data/MUC/gtt/keyword/test.json'

            with open(f'../bucketed/MUC/{split_style}/test.json', 'r') as f:
                splits = json.loads(f.read())
            
            for split_range, selections in splits.items():
                out_path = f'../ablations/MUC/{split_style}/{split_range}/{trigger_source}/{experiment}.json'
                with open('temp2.json', 'w') as f:
                    f.write(json.dumps(selections))
                create_directories_for_file(out_path)
                error_analysis_event(
                    model_out,
                    tanl_ref,
                    gtt_ref,
                    "hungarian_scorer.py",
                    "types_mapping/types_mapping_muc.json",
                    out_path,
                    roles,
                    "scorer_configs/muc_config.json",
                    True,
                    'temp2.json'
                )